# Tensors
-> They are specialized data structre similar to arrays and metrices. In Pytorch they are used to encode the input and the output of a model as well as the model parameters.

-> Tensors are similar to NumPy's ndarrays, except that tensors can run on GPUs or other specialized hardware to accelerate computing.

In [ ]:
import torch
import numpy as np

## Tensor Initialization

##### Directly from data

In [ ]:
data = [[1, 2.0], [3, 4]]
x_data = torch.tensor(data)

##### From a NumPy array
Tensors can be created from NumPy arrays and vice versa

In [ ]:
np_array = np.array(data)
x_np = torch.from_numpy(np_array)

#####From another tensor:

The new tensor retains the properties (shape, datatype) of the argument tensor, unless explicitly overridden.

In [ ]:
x_ones = torch.ones_like(x_data) # retains the properties of x_data
print(f"Ones Tensor: \n {x_ones} \n")

x_rand = torch.rand_like(x_data, dtype=torch.float) # overrides the datatype of x_data
print(f"Random Tensor: \n {x_rand} \n")

Ones Tensor: 
 tensor([[1., 1.],
        [1., 1.]]) 

Random Tensor: 
 tensor([[0.8867, 0.2708],
        [0.9708, 0.8145]]) 



#####With random or constant values:

`shape` is a tuple of tensor dimensions. In the functions below, it determines the dimensionality of the output tensor.




In [ ]:
shape = (2, 3,)
rand_tensor = torch.rand(shape)
ones_tensor = torch.ones(shape)
zeros_tensor = torch.zeros(shape)

print(f"Random Tensor: \n {rand_tensor} \n")
print(f"Ones Tensor: \n {ones_tensor} \n")
print(f"Zeros Tensor: \n {zeros_tensor}")

Random Tensor: 
 tensor([[0.5192, 0.7447, 0.4648],
        [0.6184, 0.0445, 0.1417]]) 

Ones Tensor: 
 tensor([[1., 1., 1.],
        [1., 1., 1.]]) 

Zeros Tensor: 
 tensor([[0., 0., 0.],
        [0., 0., 0.]])


##Tensor Attributes
Tensor attributes describe their shape, datatype, and the device on which they are stored.

In [ ]:
tensor = torch.rand(3, 4)

print(f"Shape of tensor: {tensor.shape}")
print(f"Datatype of tensor: {tensor.dtype}")
print(f"Device tensor is stored on: {tensor.device}")

Shape of tensor: torch.Size([3, 4])
Datatype of tensor: torch.float32
Device tensor is stored on: cpu


##Tensor Operations
Over 100 tensor operations, including transposing, indexing, slicing, mathematical operations, linear algebra, random sampling, and more are available

In [ ]:
# We move our tensor to the GPU if available
if torch.cuda.is_available():
  tensor = tensor.to('cuda')
  print(f"Device tensor is stored on: {tensor.device}")

##### Standard numpy-like indexing and slicing:



In [ ]:
tensor = torch.ones(4, 4)
tensor[:,1] = 0
print(tensor)

tensor([[1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.]])


**Joining tensors** We can use `torch.cat` or `torch.stack` to concatenate a sequence of tensors along a given dimension.


This is one of the most common confusing topics when learning tensor operations in PyTorch (or NumPy).

Let's break it down conceptually first, then use diagrams, and finally compare it with `cat`.

Here is the golden rule to remember:

> **`torch.stack` creates a NEW dimension.**
> It takes  tensors of the *exact same shape* and packages them together into a new, higher-dimensional tensor.

Let's look at your input tensor `x`. It is a 2D tensor with a shape of `(2, 3)`.

```
x shape: (2 rows, 3 columns)

      Col 0    Col 1    Col 2
Row 0 [ 0.33,    0.12,    0.23]
Row 1 [ 0.23,   -1.12,   -0.18]

```

We can visualize `x` as a single sheet of paper with a 2x3 grid written on it.

---

### Visualizing `torch.stack((x, x), dim=...)`

We are stacking two identical copies of `x`. Let's call them **Sheet A** and **Sheet B**.

#### 1. `torch.stack((x, x), dim=0)`

* **Concept:** Create a NEW dimension at index 0 (the very beginning).
* **Action:** We take the entire Sheet A and the entire Sheet B and treat them as items in a new list.
* **Analogy:** Put Sheet A on a table. Put Sheet B directly on top of it. You now have a stack of sheets.
* **Resulting Shape:** The original shape was `(2, 3)`. The new dimension (size 2, because we have 2 sheets) is inserted at the front. Result: `(2, 2, 3)`.

**Diagram:**

```text
New Dimension 0 (Depth/Batch)
       |
       |  [Sheet A: (2x3)]
       |  ---------------------
       |  | 0.33,  0.12,  0.23 |  <- Row 0
idx 0->|  | 0.23, -1.12, -0.18 |  <- Row 1
       |  ---------------------
       |
       |  [Sheet B: (2x3)]
       |  ---------------------
       |  | 0.33,  0.12,  0.23 |  <- Row 0
idx 1->|  | 0.23, -1.12, -0.18 |  <- Row 1
       |  ---------------------

```

*Look at the output in your code for dim=0. The outermost brackets contain two blocks. The first block is Sheet A, the second is Sheet B.*

---

#### 2. `torch.stack((x, x), dim=1)`

* **Concept:** Create a NEW dimension at index 1 (between rows and columns).
* **Action:** We go inside the existing Row dimension (dim 0). For *each row*, we create a stack of the corresponding rows from Sheet A and Sheet B.
* **Analogy:**
1. Cut Row 0 out of Sheet A. Cut Row 0 out of Sheet B. Stack these two strips together.
2. Cut Row 1 out of Sheet A. Cut Row 1 out of Sheet B. Stack these two strips together.
3. Put the resulting two stacks next to each other.


* **Resulting Shape:** Original `(2, 3)`. Insert new dim (size 2) at index 1. Result: `(2, 2, 3)`.

**Diagram:**

```text
Overall structure remains 2 main groups (originally rows).
Inside each group, we now have a stack of 2.

Group 0 (Original Row 0s):
   NEW Dim 1
      |
idx 0 |  [ 0.33,  0.12,  0.23 ]  (from Sheet A)
idx 1 |  [ 0.33,  0.12,  0.23 ]  (from Sheet B)

Group 1 (Original Row 1s):
   NEW Dim 1
      |
idx 0 |  [ 0.23, -1.12, -0.18 ]  (from Sheet A)
idx 1 |  [ 0.23, -1.12, -0.18 ]  (from Sheet B)

```

*Look at your code output for dim=1. You see two main blocks. The first block contains Row 0 of x followed by Row 0 of x. The second block contains Row 1 of x followed by Row 1 of x.*

---

#### 3. `torch.stack((x, x), dim=2)` (same as `dim=-1`)

* **Concept:** Create a NEW dimension at index 2 (the very end, the deepest level).
* **Action:** Go down to the individual numbers. At every position (e.g., row 0, col 0), take the number from Sheet A and the number from Sheet B and stack them into a pair.
* **Analogy:** Place Sheet A flat. Place Sheet B directly *behind* it (like looking through transparent layers). For every cell in the grid, you now see two numbers deep.
* **Resulting Shape:** Original `(2, 3)`. Insert new dim (size 2) at the end. Result: `(2, 3, 2)`.

**Diagram:**

```text
The structure looks like the original (2 rows, 3 cols),
but every single number has been replaced by a stack of 2.

Row 0:
Col0 stack: [0.33 (A), 0.33 (B)]
Col1 stack: [0.12 (A), 0.12 (B)]
Col2 stack: [0.23 (A), 0.23 (B)]

Row 1:
Col0 stack: [0.23 (A), 0.23 (B)]
Col1 stack: [-1.12 (A), -1.12 (B)]
Col2 stack: [-0.18 (A), -0.18 (B)]

```

*Look at your code output for dim=2. It looks like a 2x3 grid, but every entry is now a little pair, like `[0.3367, 0.3367]`.*

---

### The Difference Between `torch.stack` and `torch.cat`

This is crucial. They both combine tensors, but they do it differently.

1. **`torch.stack`**: Creates a **NEW** dimension. The inputs must have the *exact same shape*.
2. **`torch.cat` (concatenate)**: Joins tensors along an **EXISTING** dimension. The inputs must have the same shape *except* on the dimension you are joining along.

#### Analogy:

* **Stacking:** You have two thin paperback books. You put one on top of the other. You now have a pile of books (a new "height" dimension has been created).
* **Concatenating:** You have two thin paperback books. You tape the last page of the first book to the first page of the second book. You still have one book, just a much thicker one (the existing "page number" dimension grew).

#### Practical Comparison using your `x` tensor (Shape 2, 3)

**1. Using `torch.stack((x, x), dim=0)`**

* Inputs: Shape (2,3) and (2,3).
* Operation: Create NEW dimension 0.
* Result Shape: **(2, 2, 3)**. It's now a 3D tensor.

**2. Using `torch.cat((x, x), dim=0)`**

* Inputs: Shape (2,3) and (2,3).
* Operation: Join along EXISTING dimension 0 (rows).
* We have 2 rows, and we add another 2 rows below them.
* Result Shape: **(4, 3)**. It stays a 2D tensor, just taller.

```python
# Result of torch.cat((x, x), dim=0)
tensor([[ 0.3367,  0.1288,  0.2345], # x row 0
        [ 0.2303, -1.1229, -0.1863], # x row 1
        [ 0.3367,  0.1288,  0.2345], # x row 0 appended
        [ 0.2303, -1.1229, -0.1863]])# x row 1 appended

```

**3. Using `torch.cat((x, x), dim=1)`**

* Inputs: Shape (2,3) and (2,3).
* Operation: Join along EXISTING dimension 1 (columns).
* We have 3 columns, and we attach another 3 columns to the right side.
* Result Shape: **(2, 6)**. It stays a 2D tensor, just wider.

```python
# Result of torch.cat((x, x), dim=1)
# Row 0 of x joined with Row 0 of x
tensor([[ 0.3367,  0.1288,  0.2345,  0.3367,  0.1288,  0.2345],
# Row 1 of x joined with Row 1 of x
        [ 0.2303, -1.1229, -0.1863,  0.2303, -1.1229, -0.1863]])

```

In [ ]:
t1 = torch.cat([tensor, tensor, tensor], dim=1)
print(t1)
t2 = torch.stack([tensor, tensor, tensor], dim=1)
print(t2)

tensor([[1., 0., 1., 1., 1., 0., 1., 1., 1., 0., 1., 1.],
        [1., 0., 1., 1., 1., 0., 1., 1., 1., 0., 1., 1.],
        [1., 0., 1., 1., 1., 0., 1., 1., 1., 0., 1., 1.],
        [1., 0., 1., 1., 1., 0., 1., 1., 1., 0., 1., 1.]])
tensor([[[1., 0., 1., 1.],
         [1., 0., 1., 1.],
         [1., 0., 1., 1.]],

        [[1., 0., 1., 1.],
         [1., 0., 1., 1.],
         [1., 0., 1., 1.]],

        [[1., 0., 1., 1.],
         [1., 0., 1., 1.],
         [1., 0., 1., 1.]],

        [[1., 0., 1., 1.],
         [1., 0., 1., 1.],
         [1., 0., 1., 1.]]])


#####Multiplying tensors

In [ ]:
# This computes the element-wise product
print(f"tensor.mul(tensor) \n {tensor.mul(tensor)} \n")
# Alternative syntax:
print(f"tensor * tensor \n {tensor * tensor}")

# This computes the matrix multiplication between two tensors
print(f"tensor.matmul(tensor.T) \n {tensor.matmul(tensor.T)} \n")
# Alternative syntax:
print(f"tensor @ tensor.T \n {tensor @ tensor.T}")

tensor.mul(tensor) 
 tensor([[1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.]]) 

tensor * tensor 
 tensor([[1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.]])
tensor.matmul(tensor.T) 
 tensor([[3., 3., 3., 3.],
        [3., 3., 3., 3.],
        [3., 3., 3., 3.],
        [3., 3., 3., 3.]]) 

tensor @ tensor.T 
 tensor([[3., 3., 3., 3.],
        [3., 3., 3., 3.],
        [3., 3., 3., 3.],
        [3., 3., 3., 3.]])


##### In-place operations
Operations that have a `_` suffix are in-place. For example: `x.copy_(y)`, `x.t_()`, will change `x`.

In [ ]:
print(tensor, "\n")
tensor.add_(5)
print(tensor)

tensor([[1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.]]) 

tensor([[6., 5., 6., 6.],
        [6., 5., 6., 6.],
        [6., 5., 6., 6.],
        [6., 5., 6., 6.]])


## torch.dtype

A torch.dtype is an object that represents the data type of a torch.Tensor. PyTorch has several different data types:

### Floating point dtypes

| dtype | description |
| :---- | :---------- |
| `torch.float32` or `torch.float` | 32-bit floating point |
| `torch.float64` or `torch.double` | 64-bit floating point |
| `torch.float16` or `torch.half` | 16-bit floating point, S-E-M 1-5-10 |
| `torch.bfloat16` | 16-bit floating point, sometimes referred to as Brain floating point, S-E-M 1-8-7 |
| `torch.complex32` or `torch.chalf` | 32-bit complex with two float16 components |
| `torch.complex64` or `torch.cfloat` | 64-bit complex with two float32 components |
| `torch.complex128` or `torch.cdouble` | 128-bit complex with two float64 components |
| `torch.float8_e4m3fn` | 8-bit floating point, S-E-M 1-4-3|
| `torch.float8_e5m2` [shell] | 8-bit floating point, S-E-M 1-5-2|
| `torch.float8_e4m3fnuz` | 8-bit floating point, S-E-M 1-4-3|
| `torch.float8_e5m2fnuz` | 8-bit floating point, S-E-M 1-5-2|
| `torch.float8_e8m0fnu` | 8-bit floating point, S-E-M 0-8-0|
| `torch.float4_e2m1fn_x2` | packed 4-bit floating point, S-E-M 1-2-1|

### Integer dtypes

| dtype | description |
| :---- | :---------- |
| `torch.uint8` | 8-bit integer (unsigned) |
| `torch.int8` | 8-bit integer (signed) |
| `torch.uint16` | 16-bit integer (unsigned) |
| `torch.int16` or `torch.short` | 16-bit integer (signed) |
| `torch.uint32` | 32-bit integer (unsigned) |
| `torch.int32` or `torch.int` | 32-bit integer (signed) |
| `torch.uint64` | 64-bit integer (unsigned) |
| `torch.int64` or `torch.long` | 64-bit integer (signed) |
| `torch.bool` | Boolean |